#**Data Exploration**

In [1]:
import os

# Find the latest version of spark 4.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.1'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [517 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,159 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [998 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,264 k

In [2]:
# Import packages
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pyspark.sql.functions as F

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType


# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# Retrieves in the CSV data from Google Sheets
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSDchXr1EhgCSsxlxJ3lWPhh1kT5EJS3yv4DJ2YLeMIC3y4uq-Pp4EQknrs9zAiaI3ulne2Jyi6gR6G/pub?gid=602879552&single=true&output=csv"
response = requests.get(url)

# Write the the CSV data to a local file
with open("cvd.csv", "wb") as f:
    f.write(response.content)

# Read the local CSV file using Spark
cvd_df = spark.read.csv("cvd.csv", header=True, sep=",", inferSchema=True)

# Show DataFrame
cvd_df.show()

+--------------+--------------------+--------+-------------+-----------+------------+----------+--------+---------+------+------------+---------+---------+-----+---------------+-------------------+-----------------+----------------------------+-----------------------+
|general_health|             checkup|exercise|heart_disease|skin_cancer|other_cancer|depression|diabetes|arthritis|   sex|age_category|height_cm|weight_kg|  bmi|smoking_history|alcohol_consumption|fruit_consumption|green_vegetables_consumption|friedpotato_consumption|
+--------------+--------------------+--------+-------------+-----------+------------+----------+--------+---------+------+------------+---------+---------+-----+---------------+-------------------+-----------------+----------------------------+-----------------------+
|          Poor|Within the past 2...|      No|           No|         No|          No|        No|      No|      Yes|Female|       70-74|      150|    32.66|14.54|            Yes|                

In [4]:
# Check data types
cvd_df.describe()

DataFrame[summary: string, general_health: string, checkup: string, exercise: string, heart_disease: string, skin_cancer: string, other_cancer: string, depression: string, diabetes: string, arthritis: string, sex: string, age_category: string, height_cm: string, weight_kg: string, bmi: string, smoking_history: string, alcohol_consumption: string, fruit_consumption: string, green_vegetables_consumption: string, friedpotato_consumption: string]

In [5]:
# Convert data types
cvd_df = cvd_df.withColumn("height_cm", col("height_cm").cast(IntegerType()))
cvd_df = cvd_df.withColumn("weight_kg", col("weight_kg").cast(FloatType()))
cvd_df = cvd_df.withColumn("bmi", col("bmi").cast(FloatType()))
cvd_df = cvd_df.withColumn("alcohol_consumption", col("alcohol_consumption").cast(IntegerType()))
cvd_df = cvd_df.withColumn("fruit_consumption", col("fruit_consumption").cast(IntegerType()))
cvd_df = cvd_df.withColumn("green_vegetables_consumption", col("green_vegetables_consumption").cast(IntegerType()))
cvd_df = cvd_df.withColumn("friedpotato_consumption", col("friedpotato_consumption").cast(IntegerType()))

In [6]:
# Create temporary view
cvd_df.createOrReplaceTempView('cvd')

### **Heart Disease Prevalence by Age**

In [7]:
# Spark SQL query to calculate Heart Disease Prevalence by Age
query = """
SELECT
  age_category,
  COUNT(*) AS total_count,
  SUM(
    CASE WHEN heart_disease = 'Yes' THEN 1 ELSE 0 END
    ) AS heart_disease_count
FROM
  cvd
GROUP BY
  age_category
ORDER BY
  age_category
"""

# Execute the SQL query and store the result in a Spark DataFrame
age_df = spark.sql(query)

# Convert the Spark DataFrame to a Pandas DataFrame for plotting
age_pandas_df = age_df.toPandas()

# Add a calculated column which computes the percentage of the age group with heart disease
age_pandas_df['heart_disease_percentage'] = (age_pandas_df['heart_disease_count'] / age_pandas_df['total_count']) * 100

# Display the dataframe
age_pandas_df

,age_category,total_count,heart_disease_count,heart_disease_percentage
0,18-24,18474,93,0.503410
1,25-29,15196,113,0.743617
2,30-34,17963,193,1.074431
3,35-39,19913,258,1.295636
4,40-44,20857,412,1.975356
5,45-49,20295,651,3.207687
6,50-54,24259,1118,4.608599
7,55-59,27134,1913,7.050195
8,60-64,31268,2893,9.252271
9,65-69,32321,3691,11.419820


In [8]:
# Create an interactive line chart to visualize Heart Disease Prevalence by Age using Plotly
fig = px.line(
              age_pandas_df,
              x='age_category',
              y='heart_disease_percentage',
              markers=True,
              labels={'age_category': 'Age Category', 'heart_disease_percentage': 'Heart Disease Prevalence (%)'},
              title='<b>Heart Disease Prevalence by Age</b>',
              template='simple_white',
              )

# Customise the chart appearence
fig.update_traces(line=dict(color='#FF6961'))
fig.update_layout(title_font=dict(size=28), xaxis_title_font=dict(size=16), yaxis_title_font=dict(size=16), xaxis_tickfont=dict(size=12), yaxis_tickfont=dict(size=12))

# Show the line chart
fig.show()

The older an individual is the more of their age group will have a heart disease. This increases more exponentially after the age of 40.

### **Heart Disease Prevalence by Gender**

In [10]:
# Spark SQL query to calculate heart disease prevalence by gender
query = """
SELECT
  sex,
  COUNT(*) AS total_count,
  SUM(
    CASE WHEN heart_disease = 'Yes' THEN 1 ELSE 0 END
    ) AS heart_disease_count
FROM
  cvd
GROUP BY
  sex
ORDER BY
  sex
"""

# Execute the SQL query and store the result in a Spark DataFrame
sex_df = spark.sql(query)

# Convert the Spark DataFrame to a Pandas DataFrame for plotting
sex_pandas_df = sex_df.toPandas()

# Display the dataframe
sex_pandas_df

,sex,total_count,heart_disease_count
0,Female,153867,9492
1,Male,145445,14589


In [11]:
# Create an interactive pie chart to visualize Heart Disease Prevalence by Gender using Plotly
fig = px.pie(
             sex_pandas_df,
             names='sex',
             values='heart_disease_count',
             title='<b>Heart Disease Breakdown by Gender</b>',
             color_discrete_sequence=['#FF6961', '#DEA5A4'],
             template='simple_white'
            )

# Customise the chart appearance
fig.update_traces(textinfo='percent+label', pull=[0.1, 0], hole=0.3)
fig.update_layout(title_font=dict(size=28))


# Show the pie chart
fig.show()

Of all individuals who have heart disease, males are almost twice as likely to be at risk.